In [1]:
%reload_ext autoreload
%autoreload 2

## Imports

In [2]:
import os,sys
module_path = os.path.abspath(os.path.join(r'C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from Load import *
from Process_Features import *
from Signals_Processing import *
import numpy as np
import biosignalsnotebooks as bsnb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## Load Files

In [4]:
path = "C://Users//Rodrigo//Desktop//PhD//loop-system//Breathing Rate//Data_Analysis"

participants_data={}

for user in os.listdir(path):
    participant_path = os.path.join(path,user)
    
    if os.path.isdir(participant_path):
        participants_data[user] = {}
        for file in os.listdir(participant_path):
            if file.endswith('.xdf'):
                file_path = os.path.join(participant_path,file)
                
                condition = file.split("_")[1].split(".")[0]
                
                data = Run_files(file_path)
                
                participants_data[user][condition] = data

Stream 3: Calculated effective sampling rate 0.6189 Hz is different from specified rate 50.0000 Hz.
Stream 2: Calculated effective sampling rate 0.7198 Hz is different from specified rate 50.0000 Hz.
Stream 2: Calculated effective sampling rate 66.2452 Hz is different from specified rate 50.0000 Hz.
Stream 3: Calculated effective sampling rate 0.8801 Hz is different from specified rate 50.0000 Hz.
Stream 3: Calculated effective sampling rate 0.6201 Hz is different from specified rate 50.0000 Hz.
Stream 2: Calculated effective sampling rate 74.5119 Hz is different from specified rate 50.0000 Hz.
Stream 4: Calculated effective sampling rate 74.5324 Hz is different from specified rate 50.0000 Hz.
Stream 2: Calculated effective sampling rate 0.8285 Hz is different from specified rate 50.0000 Hz.
Stream 2: Calculated effective sampling rate 74.5283 Hz is different from specified rate 50.0000 Hz.
Stream 3: Calculated effective sampling rate 0.6351 Hz is different from specified rate 50.0000 

## Function to Load Breathing Rate

In [5]:
def Load_BreathingRate(data, stream_type: str):

    BR_Data = {}
    
    for stream in data:
        if stream["info"]["type"][0] == stream_type:
            BR_Data["Time"] = stream["time_stamps"]
            for i in range(0, int(stream["info"]["channel_count"][0])):
                BR_Data["BR"] = stream["time_series"][:, i]

    return BR_Data

## Function to rename dict key

In [6]:
def change_key(dictionary, key_to_remove, new_key):
    dictionary[new_key] = dictionary.pop(key_to_remove)

## Load Signals by condition

In [7]:
participants_signals={}
for user in participants_data:
    start_time = 0
    participants_signals[user] = getSignals(participants_data[user], "OpenSignals", "PsychoPy Markers", "PsychoPy Ratings", sensors=["ECG","EDA","RESP"])

In [8]:
participants_signals["P1"]["VR"]["Markers"].append(['end','0'])

In [9]:
participants_signals["P1"]["VR"]["Markers Timestamps"].append(np.array(participants_signals["P1"]["VR"]["Signals"]["Time"])[-1])

## Get Markers for Epochs

In [10]:
participants_epochs_markers={}
for user in participants_signals:
    print(f"User = {user}")
    epochs_markers, ratings = getEvents(participants_signals[user])
    participants_epochs_markers[user] = {"Markers": epochs_markers, "Ratings": ratings}

User = P1
User = P10
User = P11
User = P12
User = P13
User = P14
User = P15
User = P16
User = P17
User = P18
User = P19
User = P2
User = P20
User = P21
User = P22
User = P23
User = P24
User = P25
User = P26
User = P27
User = P28
User = P29
User = P3
User = P30
User = P4
User = P5
User = P6
User = P7
User = P8
User = P9


## Baseline Dataframe

In [ ]:
participants_baseline={}
for user in participants_signals:
    print("User = " + user)
    participants_baseline[user] = {}
    for condition in participants_signals[user]:
        if condition == "Baseline":
            participants_baseline[user] = nk.epochs_create(
            pd.DataFrame.from_dict(participants_signals[user][condition]["Signals"]),
            events=participants_markers[user][condition]["Onset_index"],
            sampling_rate=fs,
            epochs_start=0,
            epochs_end=participants_markers[user][condition]["EventsDiff"],
            )

In [ ]:
participants_baseline

In [ ]:
features_epochs = {}
for user in participants_baseline:
    features_epochs[user] = {}
    for condition in participants_baseline[user]:
        features_epochs[user] = getDataframe(participants_baseline[user][condition],100,16)

In [ ]:
features_epochs

## Create Epochs

In [11]:
participants_epochs_markers["P12"]["Ratings"]["VR"]["Valence"].pop(-1)
participants_epochs_markers["P12"]["Ratings"]["VR"]["Valence Timestamps"].pop(-1)
participants_epochs_markers["P12"]["Ratings"]["VR"]["Valence_index"].pop(-1)

participants_epochs_markers["P16"]["Ratings"]["VR"]["Valence"].pop(-1)
participants_epochs_markers["P16"]["Ratings"]["VR"]["Valence Timestamps"].pop(-1)
participants_epochs_markers["P16"]["Ratings"]["VR"]["Valence_index"].pop(-1)

participants_epochs_markers["P9"]["Ratings"]["VR"]["Valence"].pop(-1)
participants_epochs_markers["P9"]["Markers"]["VR"]["Offset"].pop(-1)
participants_epochs_markers["P9"]["Ratings"]["VR"]["Valence Timestamps"].pop(-1)
participants_epochs_markers["P9"]["Ratings"]["VR"]["Valence_index"].pop(-1)

69025

In [12]:
participants_epochs = {}
for user in participants_signals:
    participants_epochs[user] = {}
    print(f"User = {user}")
    for condition in participants_epochs_markers[user]["Ratings"]:
        if condition == "VR":
            valence_timestamps, arousal_timestamps = None, None

            print(f'Valence Initial Timestamps = {len(participants_epochs_markers[user]["Ratings"][condition]["Valence Timestamps"])}')
            print(f'Arousal Initial Timestamps = {len(participants_epochs_markers[user]["Ratings"][condition]["Arousal Timestamps"])}')
            
            markers = np.insert(participants_epochs_markers[user]["Ratings"][condition]["Arousal_index"], 0, participants_epochs_markers[user]["Markers"][condition]["Onset_index"][0])
            valence_values = participants_epochs_markers[user]["Ratings"][condition]['Valence']
            arousal_values = participants_epochs_markers[user]["Ratings"][condition]['Arousal']

            valence_timestamps = np.insert(participants_epochs_markers[user]["Ratings"][condition]["Valence Timestamps"], len(participants_epochs_markers[user]["Ratings"][condition]["Valence Timestamps"]), participants_epochs_markers[user]["Markers"][condition]["Offset"][-1])
            arousal_timestamps = np.insert(participants_epochs_markers[user]["Ratings"][condition]["Arousal Timestamps"], 0, participants_epochs_markers[user]["Markers"][condition]["Onset"][0])

            events_diff = valence_timestamps-arousal_timestamps
            events_diff = list(events_diff[:-1])
            print(f'Number of events diff = {len(events_diff)}')

            signals = nk.epochs_create(pd.DataFrame.from_dict(participants_signals[user][condition]["Signals"]),
                           events=markers[:-1],
                           sampling_rate=100,
                           epochs_start=0,
                           epochs_end=events_diff,
                           )
            participants_epochs[user][condition] = {
            "Signals": signals, "Arousal": arousal_values, "Valence": valence_values}

User = P1
Valence Initial Timestamps = 8
Arousal Initial Timestamps = 8
Number of events diff = 8
User = P10
Valence Initial Timestamps = 13
Arousal Initial Timestamps = 13
Number of events diff = 13
User = P11
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P12
Valence Initial Timestamps = 11
Arousal Initial Timestamps = 11
Number of events diff = 11
User = P13
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P14
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P15
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P16
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P17
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P18
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of e

In [46]:
participants_epochs = {}
for user in participants_signals:
    print("User = " + user)
    participants_epochs[user] = {}
    for condition in participants_signals[user]:
        if condition == "VR":

            valence_timestamps, arousal_timestamps = None, None

            print(f'Valence Initial Timestamps = {len(participants_markers[user][condition]["Valence Timestamps"])}')
            print(f'Arousal Initial Timestamps = {len(participants_markers[user][condition]["Arousal Timestamps"])}')
            
            markers = np.insert(participants_markers[user][condition]["Arousal_index"], 0, participants_markers[user][condition]["Onset_index"][0])
            valence_values = participants_markers[user][condition]['Valence']
            arousal_values = participants_markers[user][condition]['Arousal']

            valence_timestamps = np.insert(participants_markers[user][condition]["Valence Timestamps"], len(participants_markers[user][condition]["Valence Timestamps"]), participants_markers[user][condition]["Offset"][-1])
            arousal_timestamps = np.insert(participants_markers[user][condition]["Arousal Timestamps"], 0, participants_markers[user][condition]["Onset"][0])

            events_diff = valence_timestamps-arousal_timestamps
            events_diff = list(events_diff[:-1])
            print(f'Number of events diff = {len(events_diff)}')
            
            #Create 60 sec epochs with a sliding window of 1 sec.
            
            signals = nk.epochs_create(pd.DataFrame.from_dict(participants_signals[user][condition]["Signals"]),
                                       events=markers,
                                       sampling_rate=100,
                                       epochs_start=0,
                                       epochs_end=events_diff,
                                       )
        elif condition == "Baseline":
            events = np.arange(participants_markers[user][condition]["Onset_index"][0],
                               participants_markers[user][condition]["Offset_index"][-1]-60*100, 100)
            arousal_values = [condition for _ in range(len(events))]
            valence_values = [condition for _ in range(len(events))]
            signals = nk.epochs_create(pd.DataFrame.from_dict(participants_signals[user][condition]["Signals"]),
                                       events=events,
                                       sampling_rate=100,
                                       epochs_start=0,
                                       epochs_end=60,
                                       )
        else:
            events = np.arange(participants_markers[user][condition]["Onset_index"][0],
                               participants_markers[user][condition]["Offset_index"][-1]-60*100, 100)

            stroop_markers = [0]

            for timestamp_index in participants_markers[user][condition]["Onset_index"]:
                stroop_markers.append(
                    len(np.where(timestamp_index >= events)[0]))

            count = np.diff(stroop_markers)
            markers = [value for count, value in zip(
                count, participants_markers[user][condition]["Marker"]) for _ in range(count)]

            arousal_values = markers
            valence_values = markers

            signals = nk.epochs_create(pd.DataFrame.from_dict(participants_signals[user][condition]["Signals"]),
                                       events=events,
                                       sampling_rate=100,
                                       epochs_start=0,
                                       epochs_end=60,
                                       )

        participants_epochs[user][condition] = {
            "Signals": signals, "Arousal": arousal_values, "Valence": valence_values}

User = P1
Valence Initial Timestamps = 8
Arousal Initial Timestamps = 8
Number of events diff = 8
User = P10
Valence Initial Timestamps = 13
Arousal Initial Timestamps = 13
Number of events diff = 13
User = P11
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P12
Valence Initial Timestamps = 11
Arousal Initial Timestamps = 11
Number of events diff = 11
User = P13
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P14
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P15
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P16
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P17
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of events diff = 12
User = P18
Valence Initial Timestamps = 12
Arousal Initial Timestamps = 12
Number of e

In [16]:
participants_epochs = {}
for user in participants_signals:
    print("User = " + user)
    participants_epochs[user] = {}
    for condition in participants_signals[user]:
        if condition == "VR":

            #Get All Markers between the Onset_index (start of experiment) and the end of the experiment (last Arousal value)
            #Subtract 60 sec to obtain a window of processing before the end
            
            all_markers = np.arange(participants_markers[user][condition]["Onset_index"][0], (
                participants_markers[user][condition]["Arousal_index"][-1]-59*100), 100)

            #Keep only informative markers values - remove values between the self-report of arousal and valence. 

            for valence_index, arousal_index in zip(participants_markers[user][condition]["Valence_index"], participants_markers[user][condition]["Arousal_index"]):
                values_to_keep = (all_markers <= valence_index) | (
                    all_markers >= arousal_index)
                all_markers = all_markers[values_to_keep]
            markers = all_markers

            # Since we only have self-report every 1 min, check how many windows there are within that 1 min and attribute that self-report to the previous windows.
            
            arousal_markers = [0]

            for timestamp_index in participants_markers[user][condition]["Arousal_index"]:
                arousal_markers.append(
                    len(np.where(timestamp_index >= markers)[0]))
            markers_count = np.diff(arousal_markers)

            arousal_values = [f"{value}_{i}" for i, (count, value) in enumerate(zip(
                markers_count, participants_markers[user][condition]["Arousal"])) for _ in range(count)]
                
            valence_values = [f"{value}_{i}" for i, (count, value) in enumerate(zip(
                markers_count, participants_markers[user][condition]["Valence"])) for _ in range(count)]

            #Create 60 sec epochs with a sliding window of 1 sec.
            
            signals = nk.epochs_create(pd.DataFrame.from_dict(participants_signals[user][condition]["Signals"]),
                                       events=markers,
                                       sampling_rate=100,
                                       epochs_start=0,
                                       epochs_end=60,
                                       )
        elif condition == "Baseline":
            events = np.arange(participants_markers[user][condition]["Onset_index"][0],
                               participants_markers[user][condition]["Offset_index"][-1]-60*100, 100)
            arousal_values = [condition for _ in range(len(events))]
            valence_values = [condition for _ in range(len(events))]
            signals = nk.epochs_create(pd.DataFrame.from_dict(participants_signals[user][condition]["Signals"]),
                                       events=events,
                                       sampling_rate=100,
                                       epochs_start=0,
                                       epochs_end=60,
                                       )
        else:
            events = np.arange(participants_markers[user][condition]["Onset_index"][0],
                               participants_markers[user][condition]["Offset_index"][-1]-60*100, 100)

            stroop_markers = [0]

            for timestamp_index in participants_markers[user][condition]["Onset_index"]:
                stroop_markers.append(
                    len(np.where(timestamp_index >= events)[0]))

            count = np.diff(stroop_markers)
            markers = [value for count, value in zip(
                count, participants_markers[user][condition]["Marker"]) for _ in range(count)]

            arousal_values = markers
            valence_values = markers

            signals = nk.epochs_create(pd.DataFrame.from_dict(participants_signals[user][condition]["Signals"]),
                                       events=events,
                                       sampling_rate=100,
                                       epochs_start=0,
                                       epochs_end=60,
                                       )

        participants_epochs[user][condition] = {
            "Signals": signals, "Arousal": arousal_values, "Valence": valence_values}  # , "BR": br}

User = P1
User = P10
User = P11
User = P12
User = P13
User = P14
User = P15
User = P16
User = P17
User = P18
User = P19
User = P2
User = P20
User = P21
User = P22
User = P23
User = P24
User = P25
User = P26
User = P27
User = P28
User = P29
User = P3
User = P30
User = P4
User = P5
User = P6
User = P7
User = P8
User = P9


## Extract Features

In [13]:
features_epochs={}
# AVG_BR = []
# arousal, valence = [], []
# i = 0
for user in participants_epochs:
    print("User = " + user)
    features_epochs[user] = {}
    for condition in participants_epochs[user]:
        print("Condition = " + condition)
        features_epochs[user][condition] = {}
        if condition == "VR":
#             arousal.extend(participants_signals[user][condition]["Arousal"])
#             valence.extend(participants_signals[user][condition]["Valence"])
            print("Epochs = {}".format(len(participants_epochs[user][condition]["Signals"].keys())))
            print("Arousal = {}".format(len(participants_epochs[user][condition]["Arousal"])))
            print("Valence = {}".format(len(participants_epochs[user][condition]["Valence"])))
            for epoch in participants_epochs[user][condition]["Signals"]:
                features_epochs[user][condition][epoch] = getDataframe(participants_epochs[user][condition]["Signals"][epoch],100,16)
                # AVG_BR.append(np.mean(participants_epochs[user][condition]["BR"][epoch]["BR"]))
        else:
            print("Epochs = {}".format(len(participants_epochs[user][condition]["Signals"].keys())))
            print("Arousal = {}".format(len(participants_epochs[user][condition]["Arousal"])))
            print("Valence = {}".format(len(participants_epochs[user][condition]["Valence"])))
            for epoch in participants_epochs[user][condition]["Signals"]:
#                 arousal.append(condition)
#                 valence.append(condition)
                features_epochs[user][condition][epoch] = getDataframe(participants_epochs[user][condition]["Signals"][epoch],100,16)
                # AVG_BR.append(np.mean(participants_epochs[user][condition]["BR"][epoch]["BR"]))

User = P1
Condition = VR
Epochs = 8
Arousal = 8
Valence = 8
User = P10
Condition = VR
Epochs = 13
Arousal = 13
Valence = 13
User = P11
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P12
Condition = VR
Epochs = 11
Arousal = 11
Valence = 11
User = P13
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P14
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P15
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P16
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P17
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P18
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P19
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P2
Condition = VR
Epochs = 11
Arousal = 11
Valence = 11
User = P20
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P21
Condition = VR
Epochs = 11
Arousal = 11
Valence = 11
User = P22
Condition = VR
Epochs = 12
Arousal = 12
Valence = 12
User = P23
Condition = VR
Epochs = 12
Arousal

In [14]:
features_epochs

{'P1': {'VR': {'1':       Avg HR     Min HR     Max HR        SD    AVG RR  Minimum RR  \
   0  91.053187  84.497321  109.07836  4.903607  0.660701    0.550063   
   
      Maximum RR     SDNN    RMSSD  NN50  ...  RRV_MadBB  RRV_MCVBB  RRV_nn50  \
   0    0.710082  32.4977  15.9249     3  ...    97.8516    0.25026      11.0   
   
      RRV_nn20  RRV_pNN50  RRV_pNN20     RRV_SD1     RRV_SD2  RRV_SD2SD1  \
   0      14.0  64.705882  82.352941  190.854171  238.975928    1.252139   
   
      RRV_ApEn  
   0  0.309771  
   
   [1 rows x 74 columns],
   '2':       Avg HR     Min HR     Max HR        SD    AVG RR  Minimum RR  \
   0  90.756509  86.944117  93.736626  1.608937  0.661319    0.640091   
   
      Maximum RR     SDNN   RMSSD  NN50  ...  RRV_MadBB  RRV_MCVBB  RRV_nn50  \
   0    0.690098  11.8049  9.1957     0  ...   139.3644   0.313178      11.0   
   
      RRV_nn20  RRV_pNN50  RRV_pNN20     RRV_SD1     RRV_SD2  RRV_SD2SD1  \
   0      12.0  84.615385  92.307692  130.662458  12

In [15]:
features = features_epochs
participants = participants_epochs

## Full Dataframe

In [16]:
dataframe = pd.DataFrame(columns=features_epochs["P1"]["VR"]["1"].columns)
users, conditions, arousal, valence = [],[],[],[]
for user in features:
    print("User = " + user)
    for condition in features[user]:
        arousal.extend(participants[user][condition]["Arousal"])
        valence.extend(participants[user][condition]["Valence"])
        print("Condition = " + condition)
        for epoch in features[user][condition]:
            users.append(user)
            conditions.append(condition + "_" + epoch)
            dataframe = pd.concat([dataframe, features[user][condition][epoch]],ignore_index=True)
            
dataframe["Arousal"] = arousal
dataframe["Valence"] = valence
dataframe["User"] = users
dataframe["Condition"] = conditions
dataframe = dataframe.rename(columns={"AVG": "AVG_RSP_Rate", "Maximum": "Max_RSP_Rate", "Minimum": "Min_RSP_Rate", "STD": "STD_RSP_Rate"})

User = P1
Condition = VR
User = P10
Condition = VR
User = P11
Condition = VR
User = P12
Condition = VR
User = P13
Condition = VR
User = P14
Condition = VR
User = P15
Condition = VR
User = P16
Condition = VR
User = P17
Condition = VR
User = P18
Condition = VR
User = P19
Condition = VR
User = P2
Condition = VR
User = P20
Condition = VR
User = P21
Condition = VR
User = P22
Condition = VR
User = P23
Condition = VR
User = P24
Condition = VR
User = P25
Condition = VR
User = P26
Condition = VR
User = P27
Condition = VR
User = P28
Condition = VR
User = P29
Condition = VR
User = P3
Condition = VR
User = P30
Condition = VR
User = P4
Condition = VR
User = P5
Condition = VR
User = P6
Condition = VR
User = P7
Condition = VR
User = P8
Condition = VR
User = P9
Condition = VR


In [17]:
dataframe

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_pNN50,RRV_pNN20,RRV_SD1,RRV_SD2,RRV_SD2SD1,RRV_ApEn,Arousal,Valence,User,Condition
0,91.053187,84.497321,109.078360,4.903607,0.660701,0.550063,0.710082,32.4977,15.9249,3,...,64.705882,82.352941,190.854171,238.975928,1.252139,0.309771,Medium,Medium,P1,VR_1
1,90.756509,86.944117,93.736626,1.608937,0.661319,0.640091,0.690098,11.8049,9.1957,0,...,84.615385,92.307692,130.662458,121.587318,0.930545,0.028513,High,High,P1,VR_2
2,90.369801,85.708316,93.743471,1.711707,0.664178,0.640045,0.700049,12.6285,8.9026,0,...,79.166667,87.500000,167.888150,129.309875,0.770214,0.136369,Medium,Medium,P1,VR_3
3,91.392440,86.947491,93.740264,1.766946,0.656758,0.640066,0.690072,12.8779,10.1768,0,...,82.352941,88.235294,170.125977,143.776805,0.845120,0.108748,Low,High,P1,VR_4
4,90.339634,86.946247,96.762759,1.498535,0.664339,0.620073,0.690082,10.8007,10.9638,1,...,60.000000,80.000000,137.894101,169.216058,1.227145,0.074108,Low,High,P1,VR_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,70.119985,23.712030,249.964321,40.052733,1.043649,0.240034,2.530361,465.7141,421.9698,19,...,28.571429,42.857143,69.907952,140.029703,2.003058,0.277262,Low,Medium,P9,VR_4
348,76.649369,58.814151,222.186791,36.086855,0.857012,0.270043,1.020163,163.4676,196.2183,19,...,70.000000,70.000000,118.678721,133.629359,1.125976,0.036250,Low,Medium,P9,VR_5
349,66.322104,59.396514,74.988099,3.410516,0.907029,0.800127,1.010160,45.8703,49.4379,21,...,28.571429,78.571429,129.855837,104.148721,0.802033,0.271408,Low,Medium,P9,VR_6
350,86.065374,62.490396,260.829481,49.601103,0.809939,0.230035,0.960148,200.1834,222.3759,22,...,5.882353,58.823529,20.636083,29.307533,1.420208,0.064539,Low,Medium,P9,VR_7


In [18]:
dataframe.to_csv('.\Breathing Rate\Results\sr_dataframe.csv')

In [62]:
dataframe[dataframe["Condition"]=="PreStroop_1"]

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,Arousal,Valence,User,Condition
430,94.533360,89.537313,111.092593,4.540699,0.636040,0.540090,0.670112,28.0148,14.1445,1,...,7.0,6.0,60.000000,70.000000,NaN,124.482819,Practice_Congruente,Practice_Congruente,P1,PreStroop_1
1637,71.292829,65.923077,83.319444,3.001940,0.843039,0.720120,0.910152,34.2724,30.0050,8,...,6.0,2.0,12.500000,37.500000,NaN,25.649933,Practice_Congruente,Practice_Congruente,P10,PreStroop_1
2969,131.033444,107.125000,142.833333,10.922593,0.461337,0.420070,0.560093,41.3427,9.3875,0,...,10.0,7.0,63.636364,90.909091,NaN,188.370262,Practice_Congruente,Practice_Congruente,P11,PreStroop_1
4222,75.752801,67.404494,93.734375,5.557286,0.796105,0.640107,0.890148,55.5285,33.6148,14,...,11.0,10.0,55.555556,61.111111,NaN,116.736112,Practice_Congruente,Practice_Congruente,P12,PreStroop_1
5485,76.943024,55.036697,92.292308,12.263117,0.800967,0.650108,1.090182,133.7171,47.2114,20,...,12.0,8.0,53.333333,80.000000,NaN,81.473922,Practice_Congruente,Practice_Congruente,P13,PreStroop_1
6783,78.891233,63.147368,90.893939,8.666916,0.769997,0.660110,0.950158,86.4419,26.2849,7,...,15.0,11.0,61.111111,83.333333,NaN,89.764889,Practice_Congruente,Practice_Congruente,P14,PreStroop_1
8099,62.331503,49.991667,74.061728,4.771094,0.968265,0.810135,1.200200,74.5097,73.9018,35,...,9.0,4.0,28.571429,64.285714,NaN,156.026933,Practice_Congruente,Practice_Congruente,P15,PreStroop_1
9354,83.764266,72.277108,93.734375,5.199153,0.719132,0.640107,0.830138,45.8093,35.3082,16,...,10.0,5.0,41.666667,83.333333,NaN,104.641987,Practice_Congruente,Practice_Congruente,P16,PreStroop_1
10647,74.065678,59.396040,90.893939,7.178332,0.817748,0.660110,1.010168,79.4908,54.7122,23,...,9.0,8.0,66.666667,75.000000,NaN,113.742852,Practice_Congruente,Practice_Congruente,P17,PreStroop_1
11963,160.884382,68.954023,285.666667,61.487584,0.431322,0.210035,0.870145,162.8380,170.4503,33,...,6.0,4.0,40.000000,60.000000,NaN,88.520478,Practice_Congruente,Practice_Congruente,P18,PreStroop_1


## Visualize Signals

 ECG+EDA+RESP

In [ ]:
%matplotlib qt
colors = ["yellow", "blue","green","red"]
for participant in participants_epochs:
    if participant == "P1":
        plt.figure()
        plt.title(participant)
        hasLegend = True
        for i,condition in enumerate(participants_epochs[participant]):
            plt.text((participants_signals[participant][condition]["Markers Timestamps"][0] + participants_signals[participant][condition]["Markers Timestamps"][-1]) / 2, 6, condition, color='black', fontsize=12, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.5))
            plt.axvspan(participants_signals[participant][condition]["Markers Timestamps"][0],participants_signals[participant][condition]["Markers Timestamps"][-1],color=colors[i],alpha=0.2)
            plt.plot(participants_epochs[participant][condition]["Signals"]["1"]["Time"],participants_epochs[participant][condition]["Signals"]["1"]["ECG"],
                     c='r',label="ECG" if hasLegend else "")
            plt.plot(participants_epochs[participant][condition]["Signals"]["1"]["Time"],participants_epochs[participant][condition]["Signals"]["1"]["EDA"],
                     c='b',label="EDA" if hasLegend else "")
            plt.plot(participants_epochs[participant][condition]["Signals"]["1"]["Time"],participants_epochs[participant][condition]["Signals"]["1"]["RESP"],
                     c='k',label="RESP" if hasLegend else "")
            hasLegend = False
        plt.legend()

In [ ]:
%matplotlib qt
colors = ["yellow", "blue","green","red"]
for participant in participants_signals:
    plt.figure()
    plt.title(participant)
    hasLegend = True
    for i,condition in enumerate(participants_signals[participant]):
        plt.text((participants_signals[participant][condition]["Markers Timestamps"][0] + participants_signals[participant][condition]["Markers Timestamps"][-1]) / 2, 6, condition, color='black', fontsize=12, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.5))
        plt.axvspan(participants_signals[participant][condition]["Markers Timestamps"][0],participants_signals[participant][condition]["Markers Timestamps"][-1],color=colors[i],alpha=0.2)
        plt.plot(participants_signals[participant][condition]["Signals"]["Time"],participants_signals[participant][condition]["Signals"]["EDA"],
                 c='k',label="EDA" if hasLegend else "")
    plt.legend()

RESP

In [ ]:
def _rsp_plot_phase(rsp_signals, troughs, peaks):

    exhale_signal = pd.Series(np.full(len(rsp_signals), np.nan))
    exhale_signal[troughs] = rsp_signals["RSP_Clean"][troughs].values
    exhale_signal[peaks] = rsp_signals["RSP_Clean"][peaks].values
    exhale_signal = exhale_signal.fillna(method="backfill")

    inhale_signal = pd.Series(np.full(len(rsp_signals), np.nan))
    inhale_signal[troughs] = rsp_signals["RSP_Clean"][troughs].values
    inhale_signal[peaks] = rsp_signals["RSP_Clean"][peaks].values
    inhale_signal = inhale_signal.fillna(method="ffill")

    return exhale_signal, inhale_signal

In [ ]:
%matplotlib qt
colors = ["yellow", "blue","green","red"]
sampling_rate = 100
for participant in participants_signals:
    if participant == "P20":
        for i,condition in enumerate(participants_signals[participant]):
            rsp_signals, info = nk.rsp_process(participants_signals[participant][condition]["Signals"]["RESP"],sampling_rate=sampling_rate)
            peaks = np.where(rsp_signals["RSP_Peaks"] == 1)[0]
            troughs = np.where(rsp_signals["RSP_Troughs"] == 1)[0]
            inhale = np.where(rsp_signals["RSP_Phase"] == 1)[0]
            exhale = np.where(rsp_signals["RSP_Phase"] == 0)[0]
            
            # Create a single plot
            fig, ax = plt.subplots()
            
            # Determine unit of x-axis
            if sampling_rate is not None:
                x_axis = np.linspace(0, len(rsp_signals) / sampling_rate, len(rsp_signals))
                ax.set_xlabel("Time (seconds)",fontsize=24)
            else:
                x_axis = np.arange(0, len(rsp_signals))
                ax.set_xlabel("Samples",fontsize=24)
            ax.set_ylabel("Amplitude", fontsize=24)
            # Plot cleaned and raw respiration as well as peaks and troughs
            # ax.plot(x_axis, rsp_signals["RSP_Raw"], color="#B0BEC5", label="Raw", zorder=1)
            ax.plot(x_axis, rsp_signals["RSP_Clean"], color="#2196F3", label="Cleaned", zorder=2, linewidth=1.5)
            ax.scatter(x_axis[peaks], rsp_signals["RSP_Clean"][peaks], color="red", label="Inhalation Peaks", zorder=3)
            ax.scatter(x_axis[troughs], rsp_signals["RSP_Clean"][troughs], color="orange", label="Exhalation Troughs", zorder=4)

            # Shade region to mark inspiration and expiration.
            exhale_signal, inhale_signal = _rsp_plot_phase(rsp_signals, troughs, peaks)
        
            ax.fill_between(
                x_axis[exhale],
                exhale_signal[exhale],
                rsp_signals["RSP_Clean"][exhale],
                where=rsp_signals["RSP_Clean"][exhale] > exhale_signal[exhale],
                color="#CFD8DC",
                linestyle="None",
                label="exhalation",
            )
            ax.fill_between(
                x_axis[inhale],
                inhale_signal[inhale],
                rsp_signals["RSP_Clean"][inhale],
                where=rsp_signals["RSP_Clean"][inhale] > inhale_signal[inhale],
                color="#ECEFF1",
                linestyle="None",
                label="inhalation",
            )
        
            ax.legend(loc="upper right", fontsize=16)
            ax.tick_params(axis="both", labelsize=24)
            
            # Set titles
            ax.set_title("Filtered Signal", fontsize= 24)
            fig.suptitle("Respiration (RSP)", fontsize=24, fontweight="bold")
            
            # Show legend
            ax.legend()
            
            # Show plot
            plt.show()

Breathing Rate

In [ ]:
%matplotlib notebook
colors = ["yellow", "blue","green","red"]
for participant in participants_signals:
    plt.figure()
    plt.title(participant)
    hasLegend = True
    for i,condition in enumerate(participants_signals[participant]):
        plt.text((participants_signals[participant][condition]["Markers Timestamps"][0] + participants_signals[participant][condition]["Markers Timestamps"][-1]) / 2, 6, condition, color='black', fontsize=12, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.5))
        plt.axvspan(participants_signals[participant][condition]["Markers Timestamps"][0],participants_signals[participant][condition]["Markers Timestamps"][-1],color=colors[i],alpha=0.2)
        plt.plot(participants_signals[participant][condition]["BreathingRate"]["Time"],participants_signals[participant][condition]["BreathingRate"]["BR"],
                 c='k',label="BR" if hasLegend else "")
        hasLegend = False
    plt.legend()

ECG

In [ ]:
%matplotlib qt
colors = ["yellow", "blue","green","red"]
for participant in participants_signals:
    if participant == "P23":
        plt.figure()
        plt.title("ECG",fontsize=32)
        peaks, time = Process_ECG(participants_signals[participant][condition]["Signals"]["ECG"], 100, 16)
        hasLegend = True
        for i,condition in enumerate(participants_signals[participant]):
            # plt.text((participants_signals[participant][condition]["Markers Timestamps"][0] + participants_signals[participant][condition]["Markers Timestamps"][-1]) / 2, 6, condition, color='black', fontsize=12, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.5))
            # plt.axvspan(participants_signals[participant][condition]["Markers Timestamps"][0],participants_signals[participant][condition]["Markers Timestamps"][-1],color=colors[i],alpha=0.2)
            plt.plot(participants_signals[participant][condition]["Signals"]["Time"],participants_signals[participant][condition]["Signals"]["ECG"],
                     c='k',label="ECG" if hasLegend else "")
            plt.scatter(participants_signals[participant][condition]["Signals"]["Time"][peaks],participants_signals[participant][condition]["Signals"]["ECG"][peaks], c='r') 
            plt.xlabel("Time (seconds)", fontsize=24)
            plt.ylabel("Amplitude", fontsize=24)
            plt.xticks(fontsize=24)
            plt.yticks(fontsize=24)
            hasLegend = False
        plt.legend()

## Broken Signals

ECG+EDA+RESP

In [ ]:
from brokenaxes import brokenaxes

%matplotlib inline
colors = ["yellow", "blue","green","red"]
conditions = ["Baseline", "Pre-Stroop", "VR", "Post-Stroop"]
for participant in participants_epochs:
    # Extract all timestamps and find discontinuity
    all_timestamps = []
    for condition in participants_signals[participant]:
        all_timestamps.append(participants_signals[participant][condition]["Markers Timestamps"][0])
        all_timestamps.append(participants_signals[participant][condition]["Markers Timestamps"][-1])
    all_timestamps = sorted(all_timestamps)

    # Determine discontinuities
    discontinuities = []
    for i in range(0, len(all_timestamps), 2):
        discontinuity_start = all_timestamps[i]
        discontinuity_end = all_timestamps[i + 1]
        discontinuities.append((discontinuity_start/60, discontinuity_end/60))
    
    sorted_discontinuities = (discontinuities[0], discontinuities[3],discontinuities[1],discontinuities[2])
    
    fig = plt.figure(figsize=(18,15), dpi=600)
    hasLegend = True
    bax = brokenaxes(xlims=discontinuities, wspace=0.1, d=None)
    for i, condition in enumerate(participants_epochs[participant]):

        times = participants_epochs[participant][condition]["Signals"]["1"]["Time"]/60
        ecg = participants_epochs[participant][condition]["Signals"]["1"]["ECG"]
        eda = participants_epochs[participant][condition]["Signals"]["1"]["EDA"]
        resp = participants_epochs[participant][condition]["Signals"]["1"]["RESP"]
        
        # Plot signals
        bax.plot(times, ecg, c='r', label="ECG" if hasLegend else "")
        bax.plot(times, eda, c='b', label="EDA" if hasLegend else "")
        bax.plot(times, resp, c='k', label="RESP" if hasLegend else "")

        # Paint background for each condition
        start_time = participants_signals[participant][condition]["Markers Timestamps"][0]/60
        end_time = participants_signals[participant][condition]["Markers Timestamps"][-1]/60
        bax.axvspan(start_time, end_time, facecolor=colors[i % len(colors)], alpha=0.3)

        hasLegend = False

    for index, ax in enumerate(bax.axs):
        ticks = [round(discontinuities[index][0],1), round(discontinuities[index][1],1)]
        ax.set_xticks(ticks)
        ax.set_title(conditions[index],fontsize=24)
        ax.tick_params(axis='x', which='major', rotation=90, labelsize=24)
        ax.tick_params(axis='y', which='major', rotation=0, labelsize=24)
        
    # Add labels and a legend
    bax.set_xlabel('Time (min)',labelpad = 50, fontsize=44)
    bax.set_ylabel('Amplitude',labelpad = 50, fontsize=44)
    bax.legend()

    plt.show()
    # plt.subplots_adjust(top=0.939,
    #                     bottom=0.226,
    #                     left=0.08,
    #                     right=0.982,
    #                     hspace=0.2,
    #                     wspace=0.2)
    
    fig.savefig("G:\\O meu disco\\PhD\\Study2\\Results\\Figures\\Raw\\"+participant + ".png")


ECG

In [ ]:
from brokenaxes import brokenaxes

%matplotlib inline
colors = ["yellow", "blue","green","red"]
conditions = ["Baseline", "Pre-Stroop", "VR", "Post-Stroop"]
for participant in participants_epochs:
    # Extract all timestamps and find discontinuity
    all_timestamps = []
    for condition in participants_signals[participant]:
        all_timestamps.append(participants_signals[participant][condition]["Markers Timestamps"][0])
        all_timestamps.append(participants_signals[participant][condition]["Markers Timestamps"][-1])
    all_timestamps = sorted(all_timestamps)

    # Determine discontinuities
    discontinuities = []
    for i in range(0, len(all_timestamps), 2):
        discontinuity_start = all_timestamps[i]
        discontinuity_end = all_timestamps[i + 1]
        discontinuities.append((discontinuity_start/60, discontinuity_end/60))
    
    sorted_discontinuities = (discontinuities[0], discontinuities[3],discontinuities[1],discontinuities[2])
    
    fig = plt.figure(figsize=(18,15), dpi=600)
    hasLegend = True
    bax = brokenaxes(xlims=discontinuities, wspace=0.1, d=None)
    for i, condition in enumerate(participants_epochs[participant]):

        times = participants_epochs[participant][condition]["Signals"]["1"]["Time"]/60
        ecg = participants_epochs[participant][condition]["Signals"]["1"]["ECG"]
        
        # Plot signals
        bax.plot(times, ecg, c='k', label="ECG" if hasLegend else "")

        # Paint background for each condition
        start_time = participants_signals[participant][condition]["Markers Timestamps"][0]/60
        end_time = participants_signals[participant][condition]["Markers Timestamps"][-1]/60
        bax.axvspan(start_time, end_time, facecolor=colors[i % len(colors)], alpha=0.3)

        hasLegend = False

    for index, ax in enumerate(bax.axs):
        ticks = [round(discontinuities[index][0],1), round(discontinuities[index][1],1)]
        ax.set_xticks(ticks)
        ax.set_title(conditions[index],fontsize=24)
        ax.tick_params(axis='x', which='major', rotation=90, labelsize=24)
        ax.tick_params(axis='y', which='major', rotation=0, labelsize=24)
        
    # Add labels and a legend
    bax.set_xlabel('Time (min)',labelpad = 50, fontsize=44)
    bax.set_ylabel('Amplitude',labelpad = 50, fontsize=44)
    bax.legend()

    plt.show()

    plt.savefig("G:\\O meu disco\\PhD\\Study2\\Results\\Figures\\Raw\\ECG\\"+participant + ".png")


EDA

In [ ]:
from brokenaxes import brokenaxes

%matplotlib inline
colors = ["yellow", "blue","green","red"]
conditions = ["Baseline", "Pre-Stroop", "VR", "Post-Stroop"]
for participant in participants_epochs:
    # Extract all timestamps and find discontinuity
    all_timestamps = []
    for condition in participants_signals[participant]:
        all_timestamps.append(participants_signals[participant][condition]["Markers Timestamps"][0])
        all_timestamps.append(participants_signals[participant][condition]["Markers Timestamps"][-1])
    all_timestamps = sorted(all_timestamps)

    # Determine discontinuities
    discontinuities = []
    for i in range(0, len(all_timestamps), 2):
        discontinuity_start = all_timestamps[i]
        discontinuity_end = all_timestamps[i + 1]
        discontinuities.append((discontinuity_start/60, discontinuity_end/60))
    
    sorted_discontinuities = (discontinuities[0], discontinuities[3],discontinuities[1],discontinuities[2])
    
    fig = plt.figure(figsize=(18,15), dpi=600)
    hasLegend = True
    bax = brokenaxes(xlims=discontinuities, wspace=0.1, d=None)
    for i, condition in enumerate(participants_epochs[participant]):

        times = participants_epochs[participant][condition]["Signals"]["1"]["Time"]/60
        eda = participants_epochs[participant][condition]["Signals"]["1"]["EDA"]
        
        # Plot signals
        bax.plot(times, eda, c='k', label="EDA" if hasLegend else "")

        # Paint background for each condition
        start_time = participants_signals[participant][condition]["Markers Timestamps"][0]/60
        end_time = participants_signals[participant][condition]["Markers Timestamps"][-1]/60
        bax.axvspan(start_time, end_time, facecolor=colors[i % len(colors)], alpha=0.3)

        hasLegend = False

    for index, ax in enumerate(bax.axs):
        ticks = [round(discontinuities[index][0],1), round(discontinuities[index][1],1)]
        ax.set_xticks(ticks)
        ax.set_title(conditions[index],fontsize=24)
        ax.tick_params(axis='x', which='major', rotation=90, labelsize=24)
        ax.tick_params(axis='y', which='major', rotation=0, labelsize=24)
        
    # Add labels and a legend
    bax.set_xlabel('Time (min)',labelpad = 50, fontsize=44)
    bax.set_ylabel('Amplitude',labelpad = 50, fontsize=44)
    bax.legend()

    plt.show()

    plt.savefig("G:\\O meu disco\\PhD\\Study2\\Results\\Figures\\Raw\\EDA\\"+participant + ".png")


RESP

In [ ]:
from brokenaxes import brokenaxes

%matplotlib inline
colors = ["yellow", "blue","green","red"]
conditions = ["Baseline", "Pre-Stroop", "VR", "Post-Stroop"]
for participant in participants_epochs:
    # Extract all timestamps and find discontinuity
    all_timestamps = []
    for condition in participants_signals[participant]:
        all_timestamps.append(participants_signals[participant][condition]["Markers Timestamps"][0])
        all_timestamps.append(participants_signals[participant][condition]["Markers Timestamps"][-1])
    all_timestamps = sorted(all_timestamps)

    # Determine discontinuities
    discontinuities = []
    for i in range(0, len(all_timestamps), 2):
        discontinuity_start = all_timestamps[i]
        discontinuity_end = all_timestamps[i + 1]
        discontinuities.append((discontinuity_start/60, discontinuity_end/60))
    
    sorted_discontinuities = (discontinuities[0], discontinuities[3],discontinuities[1],discontinuities[2])
    
    fig = plt.figure(figsize=(18,15), dpi=600)
    hasLegend = True
    bax = brokenaxes(xlims=discontinuities, wspace=0.1, d=None)
    for i, condition in enumerate(participants_epochs[participant]):

        times = participants_epochs[participant][condition]["Signals"]["1"]["Time"]/60
        resp = participants_epochs[participant][condition]["Signals"]["1"]["RESP"]
        
        # Plot signals
        bax.plot(times, nk.rsp_clean(resp), c='k', label="RESP" if hasLegend else "")

        # Paint background for each condition
        start_time = participants_signals[participant][condition]["Markers Timestamps"][0]/60
        end_time = participants_signals[participant][condition]["Markers Timestamps"][-1]/60
        bax.axvspan(start_time, end_time, facecolor=colors[i % len(colors)], alpha=0.3)

        hasLegend = False

    for index, ax in enumerate(bax.axs):
        ticks = [round(discontinuities[index][0],1), round(discontinuities[index][1],1)]
        ax.set_xticks(ticks)
        ax.set_title(conditions[index],fontsize=24)
        ax.tick_params(axis='x', which='major', rotation=90, labelsize=24)
        ax.tick_params(axis='y', which='major', rotation=0, labelsize=24)
        
    # Add labels and a legend
    bax.set_xlabel('Time (min)',labelpad = 50, fontsize=44)
    bax.set_ylabel('Amplitude',labelpad = 50, fontsize=44)
    bax.legend()

    plt.show()

    plt.savefig("G:\\O meu disco\\PhD\\Study2\\Results\\Figures\\Raw\\RESP\\"+participant + ".png")
